In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
cd ..

/home/kgh6784/private/project/instance_segmentation


In [3]:
cd mmdetection

/home/kgh6784/private/project/instance_segmentation/mmdetection


In [4]:
import mmcv
import terminaltables
import pycocotools
from pycocotools.coco import COCO
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import build_dataset
from mmdet.datasets.coco import CocoDataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector
from mmdet.apis import show_result_pyplot
from mmcv.runner import load_checkpoint
from mmdet.apis import set_random_seed
import mmcv

import torch
import pylab
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# from requests import get 

# def download(url, file_name):
#     with open(file_name, "wb") as file:   
#         response = get(url)               
#         file.write(response.content)


# url = "https://download.openmmlab.com/mmdetection/v2.0/mask2former/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth"
# download(url,"mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth")

In [6]:
dataset_path = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset'

In [ ]:
#https://download.openmmlab.com/mmdetection/v2.0/mask2former/mask2former_r101_lsj_8x2_50e_coco/mask2former_r101_lsj_8x2_50e_coco_20220426_100250-c50b6fa6.pth

In [6]:
'./mask2former_r50_lsj_8x2_50e_coco.py'

'./mask2former_r50_lsj_8x2_50e_coco.py'

In [7]:
# 사용할 모데 초기 py 설정
config = './configs/mask2former/mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco.py'

# config 불러오기
cfg = mmcv.Config.fromfile(config)

# 데이터셋 지정
cfg.dataset_type = 'CocoDataset'

# data_root는 사용자가 저장한 데이터가 있는 폴더 전까지
data_root = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/'

# class 설정
cfg.classes = ('Normal',)

cfg.num_classes = 1
cfg.num_things_classes = 1
cfg.data_root = data_root 

# train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

cfg.data.samples_per_gpu=2
cfg.data.workers_per_gpu=2

cfg.data.train.data_root = data_root 
cfg.data.train.ann_file = 'modify.json' 
#cfg.data.train.ann_file = 'label(polygon)_train.json' 
cfg.data.train.img_prefix = 'train/'
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = 'modify.json' 
cfg.data.val.data_root = data_root
#cfg.data.val.ann_file = 'label(polygon)_train.json'

cfg.data.val.img_prefix = 'train/'
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = 'modify.json' 
cfg.data.test.data_root = data_root
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'test/'
cfg.data.test.classes = cfg.classes
# 클래스 수 지정
cfg.model.panoptic_head.num_things_classes=1
cfg.model.panoptic_fusion_head.num_things_classes = 1
# cfg.model.roi_head.bbox_head.num_classes = 1
# cfg.model.roi_head.mask_head.num_classes = 1
cfg.model.panoptic_head.loss_cls.class_weight = [1.0, 0.1]
# 사전 훈련 모델 지정
cfg.load_from = './mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth'
#cfg.resume_from = './work_dirs/lg_cfg2/epoch_10.pth'
# 가중치 저장 위치
cfg.work_dir = './work_dirs/lg_cfg2_m2fswinL/'

# learning rate
cfg.optimizer.lr = 0.02 / 8

# 로그 출력 시기 설정
cfg.log_config.interval = 1 
cfg.lr_config.policy = 'step'

# 평가 지표로 설정
cfg.evaluation.metric = ['bbox', 'segm']

# 평가 구간 설정

cfg.evaluation.interval = 10

# 체크포인트 구간 설정
cfg.checkpoint_config.interval = 10

# epoch 설정
cfg.runner = dict(type='EpochBasedRunner', max_epochs=20)
cfg.runner.max_epochs = 10
# 결과 재현을 위한 시드값 설정
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'



In [8]:
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Resize',
        img_scale=(1024, 1024),
        ratio_range=(0.1, 2.0),
        multiscale_mode='range',
        keep_ratio=True),
    dict(
        type='RandomCrop',
        crop_size=(1024, 1024),
        crop_type='absolute',
        recompute_bbox=True,
        allow_negative_crop=True),
    dict(
        type='FilterAnnotations', min_gt_bbox_wh=(1e-05, 1e-05), by_mask=True),
    dict(
        type='Pad',
        size=(1024, 1024),
        pad_val=dict(img=(128, 128, 128), masks=0, seg=255)),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 

In [ ]:
len(datasets[0].CLASSES)

In [8]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES

# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!


2022-07-12 04:48:15,545 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2022-07-12 04:48:16,506 - mmdet - INFO - load checkpoint from local path: ./mask2former_swin-s-p4-w7-224_lsj_8x2_50e_coco_20220504_001756-743b7d99.pth


Done (t=0.80s)
creating index...
index created!


...

2022-07-12 09:02:05,777 - mmdet - INFO - Iter [2548/2600]	lr: 2.500e-04, eta: 0:05:10, time: 1.485, data_time: 0.595, memory: 20264, loss_cls: 0.5138, loss_mask: 0.0642, loss_dice: 2.1585, d0.loss_cls: 0.6118, d0.loss_mask: 0.0647, d0.loss_dice: 2.1024, d1.loss_cls: 0.5006, d1.loss_mask: 0.0665, d1.loss_dice: 2.2152, d2.loss_cls: 0.5174, d2.loss_mask: 0.0641, d2.loss_dice: 2.2890, d3.loss_cls: 0.5104, d3.loss_mask: 0.0658, d3.loss_dice: 2.4765, d4.loss_cls: 0.5140, d4.loss_mask: 0.0651, d4.loss_dice: 2.2071, d5.loss_cls: 0.5057, d5.loss_mask: 0.0653, d5.loss_dice: 2.3956, d6.loss_cls: 0.5074, d6.loss_mask: 0.0650, d6.loss_dice: 2.3431, d7.loss_cls: 0.5051, d7.loss_mask: 0.0649, d7.loss_dice: 2.4157, d8.loss_cls: 0.5071, d8.loss_mask: 0.0651, d8.loss_dice: 2.3253, loss: 28.7723, grad_norm: 6.2017
2022-07-12 09:02:07,140 - mmdet - INFO - Iter [2549/2600]	lr: 2.500e-04, eta: 0:05:04, time: 1.363, data_time: 0.375, memory: 20264, loss_cls: 0.4855, loss_mask: 0.2966, loss_dice: 4.9243, d0.l

2022-07-12 09:02:26,967 - mmdet - INFO - Iter [2559/2600]	lr: 2.500e-04, eta: 0:04:04, time: 1.215, data_time: 0.385, memory: 20264, loss_cls: 0.6403, loss_mask: 0.3230, loss_dice: 4.5487, d0.loss_cls: 0.3339, d0.loss_mask: 0.2783, d0.loss_dice: 3.7953, d1.loss_cls: 0.6411, d1.loss_mask: 0.3147, d1.loss_dice: 4.6540, d2.loss_cls: 0.6402, d2.loss_mask: 0.3143, d2.loss_dice: 4.5689, d3.loss_cls: 0.6404, d3.loss_mask: 0.3157, d3.loss_dice: 4.5939, d4.loss_cls: 0.6406, d4.loss_mask: 0.3125, d4.loss_dice: 4.5405, d5.loss_cls: 0.6404, d5.loss_mask: 0.3140, d5.loss_dice: 4.5513, d6.loss_cls: 0.6408, d6.loss_mask: 0.3136, d6.loss_dice: 4.6427, d7.loss_cls: 0.6404, d7.loss_mask: 0.3178, d7.loss_dice: 4.6113, d8.loss_cls: 0.6403, d8.loss_mask: 0.3089, d8.loss_dice: 4.5729, loss: 54.2907, grad_norm: 4.7755
2022-07-12 09:02:28,388 - mmdet - INFO - Saving checkpoint at 2560 iterations
2022-07-12 09:02:35,833 - mmdet - INFO - Iter [2560/2600]	lr: 2.500e-04, eta: 0:03:58, time: 8.867, data_time: 0.52

2022-07-12 09:02:49,261 - mmdet - INFO - Saving checkpoint at 2570 iterations
2022-07-12 09:02:56,250 - mmdet - INFO - Iter [2570/2600]	lr: 2.500e-04, eta: 0:02:58, time: 8.358, data_time: 0.484, memory: 20264, loss_cls: 0.5338, loss_mask: 0.2314, loss_dice: 4.6082, d0.loss_cls: 0.3970, d0.loss_mask: 0.3060, d0.loss_dice: 4.0384, d1.loss_cls: 0.5346, d1.loss_mask: 0.2345, d1.loss_dice: 4.6027, d2.loss_cls: 0.5312, d2.loss_mask: 0.2339, d2.loss_dice: 4.5874, d3.loss_cls: 0.5347, d3.loss_mask: 0.2381, d3.loss_dice: 4.6294, d4.loss_cls: 0.5301, d4.loss_mask: 0.2369, d4.loss_dice: 4.5894, d5.loss_cls: 0.5390, d5.loss_mask: 0.2405, d5.loss_dice: 4.6214, d6.loss_cls: 0.5333, d6.loss_mask: 0.2381, d6.loss_dice: 4.6529, d7.loss_cls: 0.5317, d7.loss_mask: 0.2375, d7.loss_dice: 4.5882, d8.loss_cls: 0.5350, d8.loss_mask: 0.2379, d8.loss_dice: 4.5704, loss: 53.1239, grad_norm: 3.5384
2022-07-12 09:02:57,513 - mmdet - INFO - Iter [2571/2600]	lr: 2.500e-04, eta: 0:02:52, time: 1.263, data_time: 0.34

2022-07-12 09:03:16,148 - mmdet - INFO - Iter [2580/2600]	lr: 2.500e-04, eta: 0:01:58, time: 8.484, data_time: 0.377, memory: 20264, loss_cls: 0.5139, loss_mask: 0.3071, loss_dice: 4.7288, d0.loss_cls: 0.5853, d0.loss_mask: 0.4112, d0.loss_dice: 4.0511, d1.loss_cls: 0.5042, d1.loss_mask: 0.3094, d1.loss_dice: 4.7281, d2.loss_cls: 0.5067, d2.loss_mask: 0.3078, d2.loss_dice: 4.7121, d3.loss_cls: 0.5080, d3.loss_mask: 0.3081, d3.loss_dice: 4.7220, d4.loss_cls: 0.5113, d4.loss_mask: 0.3068, d4.loss_dice: 4.7136, d5.loss_cls: 0.5091, d5.loss_mask: 0.3076, d5.loss_dice: 4.6907, d6.loss_cls: 0.5047, d6.loss_mask: 0.3106, d6.loss_dice: 4.7231, d7.loss_cls: 0.5084, d7.loss_mask: 0.3070, d7.loss_dice: 4.7209, d8.loss_cls: 0.5114, d8.loss_mask: 0.3103, d8.loss_dice: 4.7337, loss: 54.8731, grad_norm: 5.8994
2022-07-12 09:03:17,241 - mmdet - INFO - Iter [2581/2600]	lr: 2.500e-04, eta: 0:01:52, time: 1.092, data_time: 0.276, memory: 20264, loss_cls: 0.8204, loss_mask: 0.2836, loss_dice: 3.6752, d0.l

RuntimeError: CUDA out of memory. Tried to allocate 764.00 MiB (GPU 0; 23.70 GiB total capacity; 18.68 GiB already allocated; 657.69 MiB free; 21.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
model

In [ ]:
cfg.num_classes